In [1]:
# импортируем необходимые библиотеки, функцию train_test_split()
# и классы SimpleImputer, StandardScaler, OneHotEncoder, 
# ColumnTransformer, LogisticRegression, Pipeline,
# функцию cross_val_score()
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

In [2]:
# записываем CSV-файл в объект DataFrame
data = pd.read_csv('Data/StateFarm_missing.csv', sep=';')
data.head(3)

,Customer Lifetime Value,Coverage,Education,EmploymentStatus,Gender,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Response
0,2763.519279,Basic,Bachelor,Employed,F,56274.0,NaN,32.0,5.0,NaN,1.0,No
1,NaN,NaN,Bachelor,Unemployed,F,0.0,NaN,13.0,42.0,NaN,NaN,No
2,NaN,NaN,NaN,Employed,F,48767.0,108.0,NaN,38.0,0.0,NaN,No


In [3]:
# создаем массив меток и массив признаков
y = data.pop('Response')

In [4]:
# создаем списки количественных 
# и категориальных столбцов
cat_columns = data.dtypes[data.dtypes == 'object'].index
num_columns = data.dtypes[data.dtypes != 'object'].index

In [5]:
# создаем конвейер для количественных переменных
num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# создаем конвейер для категориальных переменных
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

In [6]:
# создаем список трехэлементных кортежей, в котором
# первый элемент кортежа - название конвейера с
# преобразованиями для определенного типа признаков
transformers = [('num', num_pipe, num_columns),
                ('cat', cat_pipe, cat_columns)]

In [7]:
# передаем список трансформеров в ColumnTransformer
transformer = ColumnTransformer(transformers=transformers)

In [8]:
# задаем итоговый конвейер
ml_pipe = Pipeline([('transform', transformer), 
                    ('logreg', LogisticRegression(solver='lbfgs', 
                                                  max_iter=200))])


In [9]:
# выполняем перекрестную проверку, разместив итоговый 
# конвейер внутри цикла перекрестной проверки
cv_mean = cross_val_score(ml_pipe, data, y, cv=5).mean()
print('Правильность, усредненная по 5 проверочным блокам cv: {:.3f}'.format(cv_mean))

Правильность, усредненная по 5 проверочным блокам cv: 0.897
